In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os 
from datetime import datetime
import numpy as np
from matplotlib import pyplot as plt

In [4]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
#         print('end_ix',end_ix)
        out_end_ix = end_ix + n_steps_out
#         print('out_end_ix',out_end_ix)
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [5]:
# This is a example
#defin input sequence
in_seq1 = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = np.array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = np.array([in_seq1[i] + in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset)
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
for i in range(len(X)):
    print(X[i], y[i])

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
(5, 3, 3) (5, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


In [6]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import time,datetime
conn = MongoClient()
db = conn.Tracker
collection = [db.james]# db.db2, db.dn2, db.james, db.leo
clean_data = []
for col in collection:
    cursor = col.find({})
    data = [d for d in cursor]
    # print('In',str(col))
    count = 0
    for dic in data:
#         if 'o2_value' and 'hr_value' in dic.keys():
#             if (dic['longitude'] != 0) and (dic['longitude'] != '') and (dic['latitude'] != '') \
#                             and (dic['o2_value'] != '') and (dic['hr_value']!= ''):
        if 'hr_value' in dic.keys():
            if dic['hr_value']  != '':
                clean_data.append(dic)

In [7]:
# Here start the project
clean_data[0]

{'_id': ObjectId('5c9b3935c4e5a668b7e9be2c'),
 'hour': 16,
 'day': 27,
 'minute': 49,
 'latitude': '24.94519427',
 'step_value': '0',
 'year': 2019,
 'longitude': '121.37227724',
 'second': 47,
 'hr_value': '84',
 'timestamp': 1553676587.0,
 'o2_value': '95',
 'month': 3}

In [8]:
for i, line in enumerate(clean_data):
    for j, (key, value) in enumerate(line.items()):
        print(j , key, value)
    break

0 _id 5c9b3935c4e5a668b7e9be2c
1 hour 16
2 day 27
3 minute 49
4 latitude 24.94519427
5 step_value 0
6 year 2019
7 longitude 121.37227724
8 second 47
9 hr_value 84
10 timestamp 1553676587.0
11 o2_value 95
12 month 3


In [9]:
float_hr = np.zeros((len(clean_data),1))
float_o2 = np.zeros((len(clean_data),1))
float_timestamp = np.zeros((len(clean_data),1))
# dic = {3:0, 4:1, 5:2, 8:3, 11:4, 12: 5}
for i, line in enumerate(clean_data):
    for j, (key, value) in enumerate(line.items()):
        if key == 'hr_value':
            if value != '':
                float_hr[i,0] = float(value)
            else:
                float_hr[i,0]= 0
        elif key == 'o2_value':
            if value != '':
                float_o2[i,0]=float(value)
            else:
                float_o2[i,0]= 0
        elif key == 'timestamp':
            if value == 3:
                print(i, line)
                print(j,key, value)
            float_timestamp[i,0] = float(value)

In [10]:
out_seq = np.array([(float_hr[x], float_o2[x]) for x in range(len(float_o2))])
print(float_hr.shape)
print(out_seq.shape)
print(out_seq[0])

(8296, 1)
(8296, 2, 1)
[[84.]
 [95.]]


In [11]:
float_hr = float_hr.reshape((len(float_hr), 1))
float_o2 = float_o2.reshape((len(float_o2), 1))
float_timestamp = float_timestamp.reshape((len(float_o2), 1))
out_seq = out_seq.reshape((len(out_seq), 2))
dataset = np.hstack((float_hr, float_o2, out_seq))
print(dataset[:10])

[[84. 95. 84. 95.]
 [84. 95. 84. 95.]
 [84. 95. 84. 95.]
 [84. 95. 84. 95.]
 [79. 98. 79. 98.]
 [79. 98. 79. 98.]
 [79. 98. 79. 98.]
 [79. 98. 79. 98.]
 [79. 98. 79. 98.]
 [79. 98. 79. 98.]]


In [12]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
#         print('end_ix',end_ix)
        out_end_ix = end_ix + n_steps_out
#         print('out_end_ix',out_end_ix)
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :2], sequences[end_ix:out_end_ix, :2]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [13]:
timest = float_timestamp.reshape(len(float_timestamp),)
timediff = np.diff(timest)
len(timediff)
# 

8295

In [17]:
n_steps_in, n_steps_out = 3, 2
timest = float_timestamp.reshape(len(float_timestamp),)
prev = 0
for i in np.argwhere(np.diff(timest) > 600):
#     print(len(dataset[prev:i[0]+1]))
    prev = i+1
# print(len(dataset[prev:len(dataset)]))
#     X, y = split_sequences(dataset[0:i], n_steps_in, n_steps_out)
#     print(X.shape, y.shape)
#     for i in range(5, 10):
#         print(X[i], y[i])

In [18]:
timest

array([1.55367659e+09, 1.55367660e+09, 1.55367661e+09, ...,
       1.55411269e+09, 1.55411276e+09, 1.55411284e+09])

In [19]:
np.argwhere(np.diff(timest))

array([[   0],
       [   1],
       [   2],
       ...,
       [8292],
       [8293],
       [8294]])